# RAG with Langchain을 이용한 근로기준법 AI 챗봇

## 1. 근로기준법 문서를 불러와서 split 하기

In [99]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./wage_raw.docx')

splitted_chunks = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
splitted_chunks

## 2. split한 chunk들을 embedding 하기

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_upstage import UpstageEmbeddings

embedding = UpstageEmbeddings(
   model="embedding-query"
)

In [101]:
# 현재 doc를 chroma database에 embedding하여 저장하기 
from langchain_chroma import Chroma

#doc -> embedding해서 수치화한 것을 chroma DB에 저장하기
database = Chroma.from_documents(documents=splitted_chunks, embedding=embedding, collection_name='chroma-wage', persist_directory='./chroma')

In [5]:
from langchain_chroma import Chroma

database = Chroma(collection_name='chroma-wage', persist_directory='./chroma', embedding_function=embedding)

# 3. 현재 질문에 대해 유사한 문서 가져오기 (Cosine Similarity)

In [90]:
query = '근로계약서를 작성하지 않아도 되나요?'

In [102]:
retrieved_docs = database.similarity_search(query, k=2)

In [103]:
retrieved_docs

[Document(metadata={'source': './wage_raw.docx'}, page_content='② 사용자는 제1항에 따른 대통령령 중 기숙사에 관한 규정과 제99조제1항에 따른 기숙사규칙을 기숙사에 게시하거나 갖추어 두어 기숙(寄宿)하는 근로자에게 널리 알려야 한다.\n\n[제목개정 2021. 1. 5.]\n\n\n\n제2장 근로계약\n\n\n\n제15조(이 법을 위반한 근로계약) ① 이 법에서 정하는 기준에 미치지 못하는 근로조건을 정한 근로계약은 그 부분에 한정하여 무효로 한다. <개정 2020. 5. 26.>\n\n② 제1항에 따라 무효로 된 부분은 이 법에서 정한 기준에 따른다.\n\n\n\n제16조(계약기간) 근로계약은 기간을 정하지 아니한 것과 일정한 사업의 완료에 필요한 기간을 정한 것 외에는 그 기간은 1년을 초과하지 못한다.\n\n[법률 제8372호(2007. 4. 11.) 제16조의 개정규정은 같은 법 부칙 제3조의 규정에 의하여 2007년 6월 30일까지 유효함]\n\n\n\n제17조(근로조건의 명시) ① 사용자는 근로계약을 체결할 때에 근로자에게 다음 각 호의 사항을 명시하여야 한다. 근로계약 체결 후 다음 각 호의 사항을 변경하는 경우에도 또한 같다. <개정 2010. 5. 25.>\n\n1. 임금\n\n2. 소정근로시간\n\n3. 제55조에 따른 휴일\n\n4. 제60조에 따른 연차 유급휴가\n\n5. 그 밖에 대통령령으로 정하는 근로조건\n\n② 사용자는 제1항제1호와 관련한 임금의 구성항목ㆍ계산방법ㆍ지급방법 및 제2호부터 제4호까지의 사항이 명시된 서면(「전자문서 및 전자거래 기본법」 제2조제1호에 따른 전자문서를 포함한다)을 근로자에게 교부하여야 한다. 다만, 본문에 따른 사항이 단체협약 또는 취업규칙의 변경 등 대통령령으로 정하는 사유로 인하여 변경되는 경우에는 근로자의 요구가 있으면 그 근로자에게 교부하여야 한다.<신설 2010. 5. 25., 2021. 1. 5.>\n\n\n\n제18조(단시간근로자의 근로

In [ ]:
print(retrieved_docs[0].page_content)

# 4. 유사한 문서와 질문을 바탕으로 LLM에 질의하기

In [43]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [94]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()
ai_message = llm.invoke(query)

In [95]:
ai_message.content

'근로계약서는 근로자와 사용자 간의 근로관계를 명확히 하고, 근로조건과 근로계약 내용을 문서로 기록하는 것이 좋습니다. 근로계약서를 작성하지 않아도 법적으로 문제가 되지 않지만, 근로계약서를 작성하면 추후에 발생할 수 있는 분쟁을 예방하고, 근로조건과 근로계약 내용을 명확히 할 수 있습니다. 따라서, 근로계약서를 작성하는 것이 좋습니다.'

In [ ]:
from langchain import hub
from langchain.chains import RetrievalQA

prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(k=2),
    chain_type_kwargs = {"prompt":prompt}
)

In [97]:
result = qa_chain.invoke({"query": query+" 가능하다면 관련 조항도 보여주세요."})

In [98]:
result

{'query': '근로계약서를 작성하지 않아도 되나요? 가능하다면 관련 조항도 보여주세요.',
 'result': '근로계약서 작성은 필수입니다. 근로기준법 제17조에 따르면, 사용자는 근로계약을 체결할 때 근로자에게 임금, 소정근로시간, 휴일, 연차유급휴가, 그 밖에 대통령령으로 정하는 근로조건을 명시해야 합니다. 이를 위반하면 500만원 이하의 벌금에 처해질 수 있습니다.'}